In [ ]:
import os
import glob
import random
from collections import defaultdict, Counter
import pandas as pd
# pd.options.mode.copy_on_write = True
import numpy as np
from scipy.signal import resample as sci_resample
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
import librosa
from mtrf import TRF
from mtrf.stats import nested_crossval
import matplotlib.pyplot as plt
import seaborn as sns
from shapedtw.shapedtw import shape_dtw
from shapedtw.shapeDescriptors import SlopeDescriptor
import shap
from corr_shap import CorrExplainer
from pymer4.models import Lm, Lmer

sns.set_theme(
                style='white', 
                palette='Dark2',
                # rc={"figure.dpi": 150}
            )
# plt.rcParams.update({
#     "figure.facecolor": (0.0, 0.0, 0.0, 0.0),
#     "axes.facecolor": (0.0, 0.0, 0.0, 0.0),
#     "legend.facecolor": (0.0, 0.0, 0.0, 0.0),
#     "savefig.facecolor": (0.0, 0.0, 0.0, 0.0),
# })

In [ ]:
class Info:

    modality = 'va'
    sr = 30
    audio_type = 'auditory_nerve'
    trf_min_lag = 0
    trf_max_lag = 3
    
    def __init__(self):
        self.aus = ["AU12","AU14","AU15", "AU17","AU23","AU24","AU25","AU26","AU28","AU43","Pitch"]
        self.block_map = {'va': 'A->V', 'vv': 'V->V', 'vva': 'AV->V'}
        self.regularization = np.logspace(-1, 6, 10)
        self.demographics = defaultdict()
        self.trfs = defaultdict()
        self.important_aus = {
            'trf': [],
            'dtw': [],
            'lm': [],
            'glm': []
        }


    def set_demographics(self, total, gender_count, ages):
        self.demographics['N'] = f'{total} (Male={gender_count["m"]+1}, Female={gender_count["f"]})'
        self.demographics['Age'] = 'µ={:.2f}, SD={:.2f}'.format(ages.mean(), ages.std())

    
    def add_trf(self, au, trf):
        self.trfs[au] = trf

    
    def add_important_au(self, category, au, res):
        if category == 'trf' or category == 'dtw':
            self.important_aus[category].append({au: f"t({res.df})={res.statistic}, p={res.pvalue:.2f}"})
        else:
            self.important_aus[category].append({au: res})

In [ ]:
info = Info()

In [ ]:
# mapping = {
#             'M1': '1_M1_AM', 'M2': '1_M2_AM', 'M3': '1_M3_AM', 'M4': '1_M4_AM',
#             'M5': '1_M2_PM', 'M6': '1_M1_PM', 'M7': '1_M3_PM', 'M8': '1_M4_PM',
#             'M9': '2_M1_AM', 'M10': '2_M2_AM', 'M11': '2_M3_AM', 'M12': '2_M4_AM',
#             'M13': '2_M1_PM', 'M14': '2_M2_PM', 'M15': '2_M3_PM', 'M16': '2_M4_PM',
#             'F1': '1_F1_AM', 'F2': '1_F2_AM', 'F3': '1_F3_AM', 'F4': '1_F4_AM',
#             'F5': '1_F1_PM', 'F6': '1_F2_PM', 'F7': '1_F3_PM', 'F8': '2_F1_AM',
#             'F9': '2_F2_AM', 'F10': '2_F4_AM', 'F11': '2_F3_AM', 'F12': '2_F1_PM',
#             'F13': '2_F2_PM', 'F14': '2_F3_PM', 'F15': '2_F4_PM'
#         }

In [ ]:
lst_df = []
for dir in (glob.glob(f'./data/responses/*')):
    if len(os.listdir(dir)) != 0:
        files = glob.glob(dir+'/*.csv')
        for i in range(len(files)):
            if len(files) == 4:
                if i != 0:
                    temp_df = pd.read_csv(files[i])
                    lst_df.append(temp_df)
            else:
                temp_df = pd.read_csv(files[i])
                lst_df.append(temp_df)
                
df_responses = pd.concat(lst_df)

df_responses['CorrectResp'] = df_responses.Condition.apply(lambda x: 'g' if x=='TRUE' else 'h')
df_responses['Accuracy'] = df_responses.CorrectResp==df_responses.Resp
df_responses['DisplayedDyad'] = df_responses.AudioPath.str.split('/').str[4]
df_responses[['Age','Sex']] = df_responses[['Sex','Age']].where(df_responses.Subject > 5, df_responses[['Age','Sex']].values)
df_responses['VideoPath'] = df_responses.VideoPath.str.replace('\\', '/')
df_responses['Condition'] = df_responses.Condition.apply(lambda x: 'true' if x=='TRUE' else 'fake')
df_responses = df_responses.drop('Date', axis=1).reset_index(drop=True)

# Behavioral

## Demographics

In [ ]:
N = df_responses.Subject.nunique()

grouped = df_responses.groupby('Subject')
sex = grouped.Sex.apply(lambda x: x.iloc[0]).value_counts()
age = grouped.Age.apply(lambda x: x.iloc[0])

info.set_demographics(N, sex, age)

## Response counts

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
plt.tight_layout(h_pad=5, w_pad=10)
sns.histplot(x='Resp', data=df_responses, ax=axs[0, 0])
axs[0, 0].set_xticks([0, 1], labels=['Yes', 'No'])
axs[0, 0].set_xlabel('Response')
axs[0, 0].set_title('Was the video you watched a genuine interaction?')

sns.histplot(x='LikertResp', data=df_responses, bins=4, discrete=True, ax=axs[0, 1])
axs[0, 1].set_xticks([1, 2, 3, 4], labels=['1', '2', '3', '4'])
axs[0, 1].set_xlabel('Response')
axs[0, 1].set_title('How confident are you in your decision?')

sns.histplot(x='Resp', hue='Block', data=df_responses, ax=axs[1, 0], multiple='dodge', element='poly')
axs[1, 0].set_xticks([0, 1], labels=['Yes', 'No'])
axs[1, 0].set_xlabel('Response')
for t, l in zip(axs[1, 0].legend_.texts, ['A -> V', 'V -> V', 'AV -> V']):
    t.set_text(l)
axs[1, 0].set_title('Was the video you watched a genuine interaction?')
sns.move_legend(axs[1, 0], "upper left", bbox_to_anchor=(1, 1))

sns.histplot(x='LikertResp', hue='Block', data=df_responses, bins=4, discrete=True, ax=axs[1, 1], multiple='dodge', element='poly')
axs[1, 1].set_xticks([1, 2, 3, 4], labels=['1', '2', '3', '4'])
axs[1, 1].set_xlabel('Response')
for t, l in zip(axs[1, 1].legend_.texts, ['A -> V', 'V -> V', 'AV -> V']):
    t.set_text(l)
axs[1, 1].set_title('How confident are you in your decision?')
sns.move_legend(axs[1, 1], "upper left", bbox_to_anchor=(1, 1))

## Participant performance

In [ ]:
def get_stats(data=None, type=None):
    hit = len(data[(data.Condition == 'true') & (data.Resp == 'g')])
    cr = len(data[(data.Condition == 'fake') & (data.Resp == 'h')])
    miss = len(data[(data.Condition == 'true') & (data.Resp == 'h')])
    fa = len(data[((data.Condition == 'fake') & (data.Resp == 'g'))])

    if type == 'sdt':
        hit_rate = hit / (hit+miss)
        fa_rate = fa / (fa+cr)
        d_prime = stats.norm.ppf(hit_rate) - stats.norm.ppf(fa_rate)
        res = pd.Series({
                            'Hit Rate':hit_rate, 
                            'False Alarm Rate':fa_rate, 
                            'd prime':d_prime
                        })
    elif type == 'accuracy':
        res = (hit+cr)/len(data)

    return res


def ttest_between_blocks(data, block1, block2, metric):
    res = stats.ttest_rel(
        data[data.Block==block1][metric].to_numpy(), 
        data[data.Block==block2][metric].to_numpy()
    )
    print(
        f't-test between {info.block_map[block1]} & {info.block_map[block2]}: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}'
    )

### Accuracy

In [ ]:
df_acc = df_responses.groupby(['Subject', 'Block'], as_index=False).apply(lambda x: pd.Series({'Accuracy': get_stats(x, 'accuracy')}))

fig, axs = plt.subplots(ncols=2, sharey=True, figsize=(10, 4))
plt.tight_layout(w_pad=2)
sns.boxplot(y=df_acc.groupby('Subject').Accuracy.mean().to_numpy(), width=0.25, color='k', fill=False, ax=axs[0])
sns.stripplot(y=df_acc.groupby('Subject').Accuracy.mean().to_numpy(), alpha=0.3, color='k', ax=axs[0])
axs[0].set_ylim(0, 1)
axs[0].set_ylabel('Accuracy')
axs[0].axhline(y=0.5, c='k', ls='--', alpha=0.5)
axs[0].set_title('Accuracy over all participants')

res = stats.ttest_1samp(df_acc.groupby('Subject').Accuracy.mean().to_numpy(), 0.5)
print(f't-test for above chance accuracy: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}')


sns.boxplot(y='Accuracy', x='Block', hue='Block', data=df_acc, fill=False, gap=0.1)
sns.stripplot(y='Accuracy', x='Block', hue='Block', data=df_acc, alpha=0.3, legend=False)
axs[1].set_ylim(0, 1)
axs[1].set_ylabel('Accuracy')
axs[1].set_xlabel('')
axs[1].axhline(y=0.5, c='k', ls='--', alpha=0.5)
axs[1].set_xticks([0, 1, 2], list(info.block_map.values()))
axs[1].set_title('Accuracy over all participants by modality')

ttest_between_blocks(df_acc, 'va', 'vv', 'Accuracy')
ttest_between_blocks(df_acc, 'va', 'vva', 'Accuracy')
ttest_between_blocks(df_acc, 'vv', 'vva', 'Accuracy')

### SDT

In [ ]:
df_sdt = df_responses.groupby(['Subject', 'Block'], as_index=False).apply(lambda x: get_stats(x, 'sdt')).melt(
            id_vars=['Subject', 'Block'], var_name='Measure').reset_index(drop=True)

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
plt.tight_layout(w_pad=3)
sns.boxplot(y='value', data=df_sdt[df_sdt.Measure=='d prime'].groupby('Subject').mean('value'), fill=False, width=0.2, color='black', ax=axs[0])
sns.stripplot(y='value', data=df_sdt[df_sdt.Measure=='d prime'].groupby('Subject').mean('value'), alpha=0.3, dodge=True, legend=False, color='black', ax=axs[0])
axs[0].set_xticks([])
axs[0].set_xlabel('')
axs[0].set_ylabel("d'")
axs[0].axhline(y=0, linestyle='--', color='k', alpha=0.5)
axs[0].set_title("d' over all participants")

sns.boxplot(y='value', x='Measure', hue='Block', data=df_sdt, fill=False, gap=.1, ax=axs[1])
sns.stripplot(y='value', x='Measure', hue='Block', data=df_sdt, alpha=0.3, dodge=True, legend=False, ax=axs[1])
axs[1].legend_.set_title('Modality')
axs[1].legend(frameon=False)
for t, l in zip(axs[1].legend_.texts, list(info.block_map.values())):
    t.set_text(l)
axs[1].set_xlabel('')
axs[1].set_ylabel('Value')
axs[1].axhline(y=0, linestyle='--', color='k', alpha=0.5)

df_sdt = df_sdt[df_sdt.Measure=='d prime'].rename(columns={'value':'d prime'}).drop('Measure', axis=1)
res = stats.ttest_1samp(df_sdt.groupby('Subject').mean('d prime').to_numpy().flatten(), 0)
print(f't-test for above chance dprime: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}')

ttest_between_blocks(df_sdt, 'va', 'vv', 'd prime')
ttest_between_blocks(df_sdt, 'va', 'vva', 'd prime')
ttest_between_blocks(df_sdt, 'vv', 'vva', 'd prime')

## Response vs. trial duration

In [ ]:
fig, axs = plt.subplots(ncols=2, sharex=True, figsize=(10, 4))
plt.tight_layout(w_pad=3)
sns.lineplot(y='Accuracy', x='Duration', data=df_responses, ax=axs[0])
axs[0].set_ylim(0, 1)

sns.lineplot(y='LikertResp', x='Duration', data=df_responses, ax=axs[1])
axs[1].set_ylim(1, 4)
axs[1].set_ylabel('Confidence Rating');

## Distribution of dyad accuracies

In [ ]:
dyad_accuracies = df_responses.groupby('DisplayedDyad').apply(lambda x: get_stats(x, 'accuracy')).to_numpy()

ax = sns.histplot(x=dyad_accuracies, kde=True)
ax.lines[0].set_color('crimson')
ax.set_ylabel('Number of Dyads')
ax.set_title('Distribution of dyad accuracies')

# Computational

In [ ]:
df_trials = pd.read_csv('./stim/all_trials_dispDyad.csv')
df_trials = df_trials[df_trials.Modality == info.modality].reset_index(drop=True).drop('Unnamed: 0', axis='columns')

df_trials['VideoPath'] = df_trials.VideoPath.str.replace('\\', '/')
df_trials['Condition'] = df_trials.Condition.apply(lambda x: 'true' if x=='TRUE' else 'fake')

df_trials = df_trials[~df_trials.VideoPath.isin(['./stim/processed_extracts/fake/8/4_5_va.mov', './stim/processed_extracts/fake/9/1_3_va.mov'])]

## TRFs

In [ ]:
def resample_signal(signal, length):
    signal_resampled = sci_resample(signal, len(np.arange(0, length, 1/info.sr)))
    
    signal_resampled = gaussian_filter1d(signal_resampled, sigma=2)

    standardize = StandardScaler()
    signal_resampled = standardize.fit_transform(signal_resampled.reshape(-1, 1))

    return signal_resampled.flatten()


def get_data(data_type, file, length, au=None):
    match data_type:
        case 'rms':
            audio, audio_sr = librosa.load(file)
            rms_win = 0.5
            rms_hop = 1/info.sr
            rms = librosa.feature.rms(y=audio, frame_length=int(info.sr*rms_win), hop_length=int(info.sr*rms_hop))
            data=rms[0]
        case 'auditory_nerve':
            file = file.replace(os.sep, '/')
            df = pd.read_csv(file, usecols=[1])
            data = df.to_numpy().flatten()
        case 'au':
            data = pd.read_csv(file)[au].to_numpy()
        case _:
            raise ValueError(f"{data_type} not a valid value for type. Valid values are: ['rms', 'auditory_nerve', 'au']")
    
    data_resampled = resample_signal(data, length)

    return data_resampled


def prepare_input(row):
    video_path = row.VideoPath
    audio_path = row.AudioPath
    disp_dyad = row.DisplayedDyad
    duration = row.Duration
    condition = row.Condition
    au_path = f"./data/aus_pure/va/{condition}/{disp_dyad}_{video_path.split('/')[-1].replace('.mov', '_aus.csv')}"
    an_path = audio_path.replace('_audio.wav', '.csv').replace('audio', 'audio_carney')

    stims = []
    resps = []
    if os.path.exists(au_path):
        stim = get_data(data_type=info.audio_type, file=an_path, length=duration)
        for au in info.aus:
            stims.append(stim)
            resp = get_data(data_type='au', file=au_path, length=duration, au=au)
            resps.append(resp)
    
    return info.aus, stims, resps

In [ ]:
df_trials[['AU', 'Stim', 'Resp']] = df_trials.apply(lambda x: prepare_input(x), axis=1, result_type='expand')
df_trials = df_trials.explode(column=['AU', 'Stim', 'Resp'])

### Train TRF on true trials

In [ ]:
for au in info.aus:
    stims = df_trials[(df_trials.Condition=='true') & (df_trials.AU==au)].Stim
    resps = df_trials[(df_trials.Condition=='true') & (df_trials.AU==au)].Resp

    trf = TRF(direction=1)
    r_unbiased, best_regularization = nested_crossval(
        model=trf, 
        stimulus=stims, 
        response=resps, 
        fs=info.sr, 
        tmin=info.trf_min_lag, 
        tmax=info.trf_max_lag, 
        regularization=info.regularization, 
        k=5, 
        verbose=False,
        seed=1
    )
    trf.train(
                stimulus=stims, 
                response=resps, 
                fs=info.sr, 
                tmin=info.trf_min_lag, 
                tmax=info.trf_max_lag, 
                regularization=best_regularization,
                seed=1
    )
    info.add_trf(au, trf)

### Predict responses to all trials

In [ ]:
def predict_response(row):
    prediction, correlation = info.trfs[row.AU].predict(stimulus=row.Stim, response=row.Resp)
    prediction = np.array(prediction).flatten()

    return prediction, correlation

In [ ]:
df_trials[['Prediction', 'Pearsonr']] = df_trials.apply(lambda x: predict_response(x), axis=1, result_type='expand')

### Prediction accuracy True vs. Fake trials

In [ ]:
def ttest_true_fake(metric, k):
    t = []
    p = []
    for au in info.aus:
        res = stats.ttest_ind(
            df_trials[(df_trials.AU==au) & (df_trials.Condition=='true')][metric].to_list(),
            df_trials[(df_trials.AU==au) & (df_trials.Condition=='fake')][metric].to_list()
        )
        t.append(res.statistic)
        p.append(res.pvalue)
        if res.pvalue < 0.05:
            info.add_important_au(k, au, res)

    sig = ['*' if p_temp < 0.05 else 'n.s.' for p_temp in p]
    sns.barplot(x=np.asarray(info.aus), y=np.array(t), hue=sig, palette={'*': 'g', 'n.s.': 'k'})
    plt.xlabel('Action Unit')
    plt.ylabel('t')

In [ ]:
ttest_true_fake('Pearsonr', 'trf')

### Plot TRFs

In [ ]:
def plot_trf(direction, trf, channel=None, feature=None, axes=None, show=True, kind="line"):
    """
    Arguments:
        channel (None | int | str): Channel selection. If None, all channels will be used. If an integer, the channel at that index will be used. If 'avg' or 'gfp' , the average or standard deviation across channels will be computed.
        feature (None | int | str): Feature selection. If None, all features will be used. If an integer, the feature at that index will be used. If 'avg' , the average across features will be computed.
        axes (matplotlib.axes.Axes): Axis to plot to. If None is provided (default) generate a new plot.
    """
    if axes is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    else:
        fig, ax = None, axes  # dont create a new figure
    weights = trf.weights
    # select channel and or feature
    if weights.shape[0] == 1:
        feature = 0
    if weights.shape[-1] == 1:
        channel = 0
    if channel is None and feature is None:
        raise ValueError("You must specify a subset of channels or features!")
    if feature is not None:
        image_ylabel = "channel"
        if isinstance(feature, int):
            weights = weights[feature, :, :]
        elif feature == "avg":
            weights = weights.mean(axis=0)
        else:
            raise ValueError('Argument `feature` must be an integer or "avg"!')
    if channel is not None:
        image_ylabel = "feature"
        if isinstance(channel, int):
            weights = weights.T[channel].T
        elif channel == "avg":
            weights = weights.mean(axis=-1)
        elif channel == "gfp":
            weights = weights.std(axis=-1)
        else:
            raise ValueError(
                'Argument `channel` must be an integer, "avg" or "gfp"'
            )
        weights = weights.T  # transpose so first dimension is time
    # plot the result
    scaler = StandardScaler()
    if kind == "line":
        # ax.plot(
        #     trf.times.flatten(), scaler.fit_transform(weights.reshape(-1, 1)), linewidth=2 - 0.01 * weights.shape[-1]
        # )
        ax.plot(
            trf.times.flatten(), scaler.fit_transform(weights.reshape(-1, 1)), linewidth=2, color='k'
        )
        ax.set(
            xlabel="Time lag[s]",
            ylabel="Amplitude [a.u.]",
            xlim=(trf.times.min(), trf.times.max()),
            ylim=(-2.5, 2.5)
        )
        ax.axhline(y=0, color='k', linestyle='--', alpha=0.25)
    if show is True:
        plt.show()
    if fig is not None:
        return fig

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplots_adjust(hspace=1, wspace=0.5)
for i, (au, trf) in enumerate(info.trfs.items()):
    ax = plt.subplot(4, 3, i + 1)
    plot_trf(direction=1, trf=trf, axes=ax, show=False) 
    ax.set_title(au)
    ax.get_lines()[0].set_color("k")

## DTW

In [ ]:
def dtw_distance(row):
    # define the shape descriptor to use for DTW
    slope_descriptor = SlopeDescriptor(slope_window=5)
    # DTW of the two time series
    shape_dtw_dependent_results = shape_dtw(
        x=row.Resp,
        y=row.Prediction,
        subsequence_width=30, # so this means for the matching that we take about 10 indices into account
        shape_descriptor=slope_descriptor
    )
    dependent_distance = round(shape_dtw_dependent_results.distance, 2)
    # normalize distance (squash between 0 and 1)
    dependent_distance_normalized = shape_dtw_dependent_results.normalized_distance

    return dependent_distance

In [ ]:
df_trials['DTWDistance'] = df_trials.apply(lambda x: dtw_distance(x), axis=1)

### Distance between True and Fake trials

In [ ]:
ttest_true_fake('DTWDistance', 'dtw')

## SVM

In [ ]:
def get_training_data(group):
    feature = group.Pearsonr.to_numpy()
    target = group.Condition.to_numpy()[0]
    trial = group.VideoPath.to_numpy()[0]
    
    return feature, target, trial


def nested_cv(features, targets):
	N_TRIALS = 10
	scores = np.zeros(N_TRIALS)

	svc = SVC(probability=True)
	param_grid = [{'C': np.logspace(-5, 3, 9), 'kernel':['linear']}]

	for i in range(N_TRIALS):
		inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
		outer_cv = KFold(n_splits=3, shuffle=True, random_state=i)

		model = GridSearchCV(estimator=svc, param_grid=param_grid, cv=inner_cv)
		model.fit(features, targets)

		test_score = cross_val_score(model, features, targets, cv=outer_cv)
		scores[i] = test_score.mean()

	return scores

In [ ]:
svm_data = []
df_trials.groupby('VideoPath').apply(lambda x: svm_data.append(get_training_data(x)))

features, targets, ids = map(list, zip(*svm_data))
# Standardize features (i.e. - Pearson r values) before training SVM on it
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, targets, test_size=0.5, random_state=1)

In [ ]:
scores = nested_cv(scaled_features, targets)

In [ ]:
print(f'Unbiased accuracy (using nested cross-validation): µ={np.asarray(scores).mean():.2f}, SD={np.asarray(scores).std():.2f}')
res = stats.ttest_1samp(scores, 0.5)
print(f't-test for above chance performance: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}')

sns.boxplot(data=scores, width=0.25, fill=False, color='k')
sns.stripplot(data=scores, alpha=0.25, color='k')
plt.axhline(y=0.5, color='k', ls='--')
plt.ylabel('SVM accuracy')
plt.xlabel('')
plt.xticks([])
plt.ylim(0, 1);

In [ ]:
svc = SVC(probability=True)
param_grid = [{'C': np.logspace(-5, 3, 9), 'kernel':['linear']}]
cv = KFold(n_splits=5, shuffle=True, random_state=1)
model = GridSearchCV(estimator=svc, param_grid=param_grid, cv=cv, n_jobs=-1)
model.fit(X_train, y_train)

final_clf = SVC(C=model.best_params_.get('C'), kernel='linear', probability=True)
final_clf.fit(X_train, y_train)
print(f'SVM accuracy: {final_clf.score(X_test, y_test):.2f}')

### Shapley values on SVM

In [ ]:
shap.initjs()

model_labels = final_clf.classes_
true_label_idx = np.argwhere(model_labels=='true')[0][0]
fake_label_idx = np.argwhere(model_labels=='fake')[0][0]

#### Explainer

In [ ]:
explainer = shap.Explainer(final_clf.predict_proba, X_train, feature_names=info.aus)
shap_values = explainer(scaled_features)
shap.plots.beeswarm(shap_values[:, :, true_label_idx], max_display=11, show=False);

#### CorrExplainer

In [ ]:
ex_empirical = CorrExplainer(final_clf.predict_proba, X_train, sampling="empirical", feature_names=info.aus)
shap_empirical = ex_empirical(scaled_features)
shap.plots.beeswarm(shap_empirical[:, :, true_label_idx], show=False, max_display=11)

# Behavioral x Computational

In [ ]:
def combine_dfs(row):
    trial = df_trials[df_trials.VideoPath==row.VideoPath]
    aus = trial.AU.to_numpy()
    rs = trial.Pearsonr.to_numpy()

    return aus, rs
            

df = df_responses[(df_responses.Block == info.modality) & (~df_responses.VideoPath.isin(['./stim/processed_extracts/fake/8/4_5_va.mov', './stim/processed_extracts/fake/9/1_3_va.mov']))]
df[['AU', 'Pearsonr']] = df.apply(lambda x: combine_dfs(x), axis=1, result_type='expand')
df = df.explode(column=['AU', 'Pearsonr']).reset_index(drop=True)
df = df.astype({'Resp': 'category', 'Pearsonr': float})

## LM

In [ ]:
df_lm = df.groupby(['VideoPath', 'AU'], as_index=False).apply(
    lambda x: pd.Series({
        'Pearsonr': x.Pearsonr.mean(),
        'Genuineness': Counter(x.Resp)['g']/len(x)
    }))

In [ ]:
for au in info.aus:
    model = Lm("Genuineness ~ Pearsonr", data=df_lm[df_lm.AU==au])
    model.fit(summarize=False)
    if model.coefs.loc['Pearsonr']['P-val'] < 0.05:
        print('**********************************************************************')
        print(au)
        print(model.fit())
        info.add_important_au('lm', au, model.fit())

## GLM (logit)

In [ ]:
for au in info.aus:
    model = Lmer("Resp ~ Pearsonr + (1|Subject)", family='binomial', data=df[df.AU==au])
    model.fit(summarize=False)
    if model.coefs.loc['Pearsonr']['P-val'] < 0.05:
        print('**********************************************************************')
        print(au)
        print(model.fit)
        info.add_important_au('glm', au, model.fit())

## TRF fit vs. SDT category

In [ ]:
def mode_response(data):
    mode = data.Resp.mode()
    if len(mode) != 1:
        confidence = data.groupby('Resp')['LikertResp'].mean()
        
        confidence_in_true = confidence['g']
        confidence_in_fake = confidence['h']
        if confidence_in_true > confidence_in_fake:
            return 'g'
        elif confidence_in_true < confidence_in_fake:
             return 'h'
        else:
            # arbitrary decision (doesn't make a huge amount of difference if 'h' 
            # or a random choice between 'g' and 'h')
            return 'g'
    else:
        return mode.values[0]


def sdt_cat(pair):
    match pair:
        case ('true', 'g'):
            return 'hit'
        case ('true', 'h'):
            return 'miss'
        case ('fake', 'g'):
            return 'fa'
        case ('fake', 'h'):
            return 'cr'


df_final = df.groupby(['VideoPath', 'AU'], as_index=False).apply(
    lambda x: pd.Series({
        'Condition': x.Condition.iloc[0],
        'ModeResp': mode_response(x),
        'Pearsonr': x.Pearsonr.mean()
    }))
df_final['SDT'] = df_final.apply(lambda x: sdt_cat((x.Condition, x.ModeResp)), axis=1)

### Participants

#### AU25 vs. AU43 for Hit, Miss, CR & FA

In [ ]:
sns.barplot(
                data=df_final[(df_final.AU=='AU25') | (df_final.AU=='AU43')],
                x='SDT',
                y='Pearsonr',
                hue='AU',
                order=['hit', 'miss', 'cr', 'fa'],
                fill=False,
                gap=0.15
            )
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.75)
plt.ylabel('TRF fits')
plt.xlabel('')
plt.ylim(-0.15, 0.2);


for cat in df_final.SDT.unique():
    temp = df_final[df_final.SDT==cat]
    temp_aus = temp[(temp.AU=='AU25') | (temp.AU=='AU43')]

    res = stats.ttest_rel(
        temp_aus[temp_aus.AU=='AU25'].Pearsonr.to_numpy(), 
        temp_aus[temp_aus.AU=='AU43'].Pearsonr.to_numpy()
    )
    print(f't-test between AU25 & AU43 for {cat}: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}')

#### AU25 vs. AU43 for True & Fake

In [ ]:
sns.barplot(
                data=df_final[(df_final.AU=='AU25') | (df_final.AU=='AU43')],
                x='Condition',
                y='Pearsonr',
                hue='AU',
                fill=False,
                gap=0.15
            )
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.75)
plt.ylabel('TRF fits')
plt.xlabel('')
plt.ylim(-0.15, 0.2);

for cond in df_final.Condition.unique():
    temp = df_final[df_final.Condition==cond]
    temp_aus = temp[(temp.AU=='AU25') | (temp.AU=='AU43')]

    res = stats.ttest_rel(
        temp[temp.AU=='AU25'].Pearsonr.to_numpy(), 
        temp[temp.AU=='AU43'].Pearsonr.to_numpy()
    )
    print(f't-test between AU25 & AU43 for {cond}: t({res.df})={res.statistic:.2f}, p={res.pvalue:.2f}')

#### Eyes vs. Mouth (combined AUs) for Hit, Miss, CR & FA

In [ ]:
df = df[df.AU.isin(['AU12', 'AU15', 'AU17', 'AU25', 'AU43'])]
df['region'] = df.apply(lambda x: 'eyes' if x.AU=='AU43' else 'mouth', axis=1)

df_final = df.groupby(['VideoPath', 'region'], as_index=False).apply(
    lambda x: pd.Series({
        'Condition': x.Condition.iloc[0],
        'ModeResp': mode_response(x),
        'Pearsonr': x.Pearsonr.mean()
    }))
df_final['SDT'] = df_final.apply(lambda x: sdt_cat((x.Condition, x.ModeResp)), axis=1)
df_final

In [ ]:
sns.barplot(
                data=df_final,
                x='SDT',
                y='Pearsonr',
                hue='region',
                order=['hit', 'miss', 'cr', 'fa'],
                hue_order=['mouth', 'eyes'],
                fill=False,
                gap=0.15
            )
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.75)
plt.ylabel('TRF fits')
plt.xlabel('')
plt.ylim(-0.15, 0.2);

In [ ]:
# import pickle

# with open('main_results.pkl', 'wb') as f:
#     pickle.dump(info, f)

# with open('main_results.pkl', 'rb') as f:
#     test = pickle.load(f)

### Model

In [ ]:
# sns.barplot(
#                 data=df_last[(df_last['AU']=='AU25') | (df_last['AU']=='AU43')],
#                 x='SVM_SDT',
#                 y='Pearsonr',
#                 hue='AU',
#                 order=['hit', 'miss', 'cr', 'fa'],
#                 fill=False,
#                 gap=0.15
#             )
# plt.axhline(y=0, color='k', linestyle='--', linewidth=0.75)
# plt.ylabel('TRF fits')
# plt.xlabel('');

# Notes

- Focusing on eyes appears to be a sub-optimal strategy. Changing focus to the mouth might lead to fewer false alarms (and perhaps also fewer misses).

============================================================================================================================================================

- AU25 might seem like a weird AU to be important in judging genuineness, but it is important to note that it most often co-occurs with AU12 (smile).
  - Co-occurring AUs:
      - AU17: 15
      - AU25: 12

============================================================================================================================================================

- The reliability of the mouth region compared to the eyes in judging genuineness of interactions can be explained by the fact that mouth movements are more discrete and occur in response to specific events in the Speaker's speech. Blinks, on the other hand, include physiological 'noise' in addition to socially relevant cues. Moreover, cues like smiles are higher-level concepts representing social percepts concretely while the role of blinks and gaze direction is more implicit and low-level such that many people are unaware of the influence the eyes have on social interactions.